# Private Predictions with TFE Keras

# Step 3: Private Prediction using TFE Keras - Serving (Client)

After training your model with normal Keras and securing it with TFE Keras, you are ready to request some private predictions.

In [ ]:
import numpy as np
import tensorflow as tf
import tf_encrypted as tfe

from tensorflow.keras.datasets import mnist

## Data

Here, we preprocess our MNIST data. This is identical to how we preprocessed during training.

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Set up `tfe.serving.QueueClient`


Before querying the model, we just have to connect to it. To do so, we can create a client with `tfe.serving.QueueClient`. This creates a TFE queueing server on the client side that connects to the queueing server set up by `tfe.serving.QueueServer` in **Secure Model Serving**. The queue will be responsible for secretly sharing the plaintext data before submitting the shares in a prediction request.

Note that we have to use the same configuration as used by the server, including player configuration and protocol.

In [ ]:
config = tfe.RemoteConfig.load("/tmp/tfe.config")

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

In [ ]:
input_shape = (1, 28, 28, 1)
output_shape = (1, 10)

In [ ]:
client = tfe.serving.QueueClient(
    input_shape=input_shape,
    output_shape=output_shape)

In [ ]:
sess = tfe.Session(config=config)

## Query Model

You are ready to get some private predictions! Calling `client.run` will insert the image into the queue created above, secret share the data locally, and submit the shares to the model server in **Secure Model Serving**.

In [9]:
# User inputs
num_tests = 100
images, expected_labels = x_test[100:num_tests+100], y_test[100:num_tests+100]

In [10]:
for image, expected_label in zip(images, expected_labels):
    
    res = client.run(
        sess,
        image.reshape(1, 28, 28, 1))
    
    predicted_label = np.argmax(res)
    
    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "incorrectly",
        predicted_label))

The image had label 6 and was correctly classified as 6
The image had label 0 and was correctly classified as 0
The image had label 5 and was correctly classified as 5
The image had label 4 and was correctly classified as 4
The image had label 9 and was correctly classified as 9
The image had label 9 and was correctly classified as 9
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1
The image had label 9 and was correctly classified as 9
The image had label 4 and was correctly classified as 4
The image had label 8 and was correctly classified as 8
The image had label 7 and was correctly classified as 7
The image had label 3 and was correctly classified as 3
The image had label 9 and was correctly classified as 9
The image had label 7 and was correctly classified as 7
The image had label 4 and was correctly classified as 4
The image had label 4 and was correctly classified as 4
The image had label 4 and was correctly classifi

We are able to classify these three images correctly! But what's special about these predictions is that we haven't revealed any private information to get this service. The model host never saw your input data or your predictions, and you never downloaded the model. You were able to get private predictions on encrypted data with an encrypted model!